In [4]:
!wget -O raw.csv -nc "http://www.belpa.be/pls/pub/belpa.csv?piv_naam=&pin_begr_jr=&piv_postcode=&piv_gemeente=&piv_subtype_bedrag=&piv_type_bedrag=&pin_ondergrens=&pin_bovengrens=&pin_pagesize=10&pin_page_start=1&piv_sort=sort_webnaam&pin_sort_dir=0&piv_lng=EN"

--2016-05-05 19:38:39--  http://www.belpa.be/pls/pub/belpa.csv?piv_naam=&pin_begr_jr=&piv_postcode=&piv_gemeente=&piv_subtype_bedrag=&piv_type_bedrag=&pin_ondergrens=&pin_bovengrens=&pin_pagesize=10&pin_page_start=1&piv_sort=sort_webnaam&pin_sort_dir=0&piv_lng=EN
Resolving www.belpa.be... 195.244.170.200
Connecting to www.belpa.be|195.244.170.200|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/vnd.ms-excel]
Saving to: 'raw.csv'

raw.csv                 [               <=>  ]  23.02M  2.08MB/s    in 10s     

2016-05-05 19:39:08 (2.31 MB/s) - 'raw.csv' saved [24134684]



In [1]:
import pandas as pd

In [45]:
df_raw = pd.read_csv('raw.csv', delimiter=';', encoding='utf-8')
df_raw.head()

,name,postal_code,municipality,eagf,eafr,total,year,I.1,I.2,I.3,...,V/B.4.2,V/B.4.3,V/B.4.4,V/B.4.5,V/B.5,V/B.6,VI/A.1,VI/B.1,VI/C.1,VI/D.1
0,"LTC ""DE PASTORIJHOEVE""",3990,Peer,"9.511,50","231,00","9.742,50",2014,"9.511,50","0,00","0,00",...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
1,' T KALVERHOF,2360,Oud-Turnhout,"10.188,19","0,00","10.188,19",2014,"10.188,19","0,00","0,00",...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
2,' T KRISTOFFELHOF,2440,Geel,"1.038,59","0,00","1.038,59",2014,"1.038,59","0,00","0,00",...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
3,''BPOST'',1000,Brussel/Bruxelles,"1.580,04","0,00","1.580,04",2014,"0,00","0,00","0,00",...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
4,'S JONGERS DIRK,2630,Aartselaar,"8.233,32","535,36","8.768,68",2014,"8.171,31","0,00","0,00",...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"


In [46]:
id_vars = 'name	postal_code	municipality	eagf	eafr	total	year'.split('\t')
all_columns = list(df_raw.columns)
total_cols = ['total', 'eafr', 'eagf']
value_vars = list(set(all_columns) - set(id_vars))

In [47]:
def fix_nums(series):
    return series.astype(str).str.replace('.', '').str.replace(',', '.')

for val in value_vars + total_cols:
    df_raw[val] = pd.to_numeric(fix_nums(df_raw[val]), errors='coerce')
    
df_raw.head()

,name,postal_code,municipality,eagf,eafr,total,year,I.1,I.2,I.3,...,V/B.4.2,V/B.4.3,V/B.4.4,V/B.4.5,V/B.5,V/B.6,VI/A.1,VI/B.1,VI/C.1,VI/D.1
0,"LTC ""DE PASTORIJHOEVE""",3990,Peer,9511.50,231.00,9742.50,2014,9511.50,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,' T KALVERHOF,2360,Oud-Turnhout,10188.19,0.00,10188.19,2014,10188.19,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,' T KRISTOFFELHOF,2440,Geel,1038.59,0.00,1038.59,2014,1038.59,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,''BPOST'',1000,Brussel/Bruxelles,1580.04,0.00,1580.04,2014,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,'S JONGERS DIRK,2630,Aartselaar,8233.32,535.36,8768.68,2014,8171.31,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
df = pd.melt(df_raw, id_vars=id_vars, value_vars=value_vars, value_name='amount', var_name='scheme')
df = df[df['amount'] > 0]
df = df.drop(total_cols, 1)
df.head()

,name,postal_code,municipality,year,scheme,amount
83614,A.BERGMANS S. AGR.,7131,Binche,2014,IV/A.15,2655.0
83646,ABRASSART FREDDY,7370,Dour,2014,IV/A.15,13456.4
83647,ABRASSART HENRY,7370,Dour,2014,IV/A.15,1043.6
83648,ABSHOFF CEDRIC,4890,Thimister-Clermont,2014,IV/A.15,3532.0
83656,ACHENNE ANDRE-MARIE,6681,Sainte-Ode,2014,IV/A.15,148.5


In [49]:
df = df.rename(columns={
    'name': 'recipient_name',
    'postal_code': 'recipient_postcode',
    'municipality': 'recipient_location',
})
df['currency'] = 'EUR'
df['country'] = 'BE'

def generate_id(row):
    return u'BE-%s-%s' % (row['recipient_postcode'], row['recipient_name'])

df['recipient_id'] = df.apply(generate_id, 1)
df.head()

,recipient_name,recipient_postcode,recipient_location,year,scheme,amount,currency,country,recipient_id
83614,A.BERGMANS S. AGR.,7131,Binche,2014,IV/A.15,2655.0,EUR,BE,BE-7131-A.BERGMANS S. AGR.
83646,ABRASSART FREDDY,7370,Dour,2014,IV/A.15,13456.4,EUR,BE,BE-7370-ABRASSART FREDDY
83647,ABRASSART HENRY,7370,Dour,2014,IV/A.15,1043.6,EUR,BE,BE-7370-ABRASSART HENRY
83648,ABSHOFF CEDRIC,4890,Thimister-Clermont,2014,IV/A.15,3532.0,EUR,BE,BE-4890-ABSHOFF CEDRIC
83656,ACHENNE ANDRE-MARIE,6681,Sainte-Ode,2014,IV/A.15,148.5,EUR,BE,BE-6681-ACHENNE ANDRE-MARIE


In [50]:
df.year.value_counts()

2014    87489
Name: year, dtype: int64

In [52]:
df.to_csv('be_2014.csv', index=False, encoding='utf-8')